<a href="https://colab.research.google.com/github/isha-ingersol/final-year-project/blob/main/final_year_project_draft_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab Notebook for Analysing Motion Data

## Step 1: Environment Setup

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

# Optional: Mount Google Drive (uncomment the lines below if needed in Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step 2: Data Loading

In [4]:
# Define the function to load JSON file into a DataFrame
def load_motion_data(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Check the structure of the JSON file
    if 'frames' not in data or not isinstance(data['frames'], list):
        print(f"Unexpected JSON structure in file: {json_file}")
        return pd.DataFrame()  # Return an empty DataFrame for invalid files

    # Flatten JSON structure into a DataFrame
    frames = []
    for frame in data['frames']:
        if 'keypoints' in frame and 'timestamp' in frame:
            frame_data = {f"keypoint_{i}": frame['keypoints'][i] for i in range(len(frame['keypoints']))}
            frame_data['timestamp'] = frame['timestamp']
            frames.append(frame_data)
        else:
            print(f"Skipping frame due to missing data: {frame}")

    return pd.DataFrame(frames)

# Define the directory containing the motion data JSON files
motion_data_dir = '/content/drive/MyDrive/final-year-project/motion_data/'

# Load all JSON files in the directory
motion_data_files = [f for f in os.listdir(motion_data_dir) if f.endswith('.json')]

# Combine data from all JSON files into a single DataFrame
all_motion_data = pd.DataFrame()

for json_file in motion_data_files:
    file_path = os.path.join(motion_data_dir, json_file)
    motion_df = load_motion_data(file_path)
    if not motion_df.empty:
        motion_df['source_file'] = json_file  # Add a column to track the source file
        all_motion_data = pd.concat([all_motion_data, motion_df], ignore_index=True)

# Display a sample of the combined DataFrame
print("Combined Motion Data Sample:")
print(all_motion_data.head())

Unexpected JSON structure in file: /content/drive/MyDrive/final-year-project/motion_data/processed_processed_temp_All_day.json
Unexpected JSON structure in file: /content/drive/MyDrive/final-year-project/motion_data/processed_processed_temp_Blow_sth_up.json
Unexpected JSON structure in file: /content/drive/MyDrive/final-year-project/motion_data/processed_processed_temp_Beer.json
Unexpected JSON structure in file: /content/drive/MyDrive/final-year-project/motion_data/processed_processed_temp_Bus.json
Unexpected JSON structure in file: /content/drive/MyDrive/final-year-project/motion_data/processed_processed_temp_Belittle.json
Unexpected JSON structure in file: /content/drive/MyDrive/final-year-project/motion_data/processed_processed_temp_Bed.json
Unexpected JSON structure in file: /content/drive/MyDrive/final-year-project/motion_data/processed_processed_temp_A_turn_goes_to.json
Unexpected JSON structure in file: /content/drive/MyDrive/final-year-project/motion_data/processed_processed_t

## Step 3: Keypoints Mapping

In [5]:
# Adjust keypoint indices based on supervisor's note
keypoint_mapping = {
    112: "Right Hand Wrist",  # Adjusted index (113 - 1)
    11: "Right Elbow",
    10: "Right Shoulder",
    # Add other mappings as needed
}

# Example: Extract keypoint data for visualisation
keypoint_columns = [f"keypoint_{idx}" for idx in keypoint_mapping.keys()]
keypoint_data = all_motion_data[keypoint_columns + ['timestamp']]

# Rename columns for clarity
keypoint_data.rename(columns={f"keypoint_{idx}": name for idx, name in keypoint_mapping.items()}, inplace=True)
print("Keypoint Data Sample:")
print(keypoint_data.head())

KeyError: "None of [Index(['keypoint_112', 'keypoint_11', 'keypoint_10', 'timestamp'], dtype='object')] are in the [columns]"

## Step 4: Visualisation

In [ ]:
# Plot motion trajectories of specific keypoints over time
def plot_keypoint_motion(data, keypoint_name):
    plt.figure(figsize=(10, 6))
    plt.plot(data['timestamp'], data[keypoint_name], label=keypoint_name, marker='o')
    plt.title(f"Motion Trajectory of {keypoint_name}")
    plt.xlabel("Time (s)")
    plt.ylabel("Position Value")
    plt.legend()
    plt.grid()
    plt.show()

# Example: Plot motion of the Right Hand Wrist
plot_keypoint_motion(keypoint_data, "Right Hand Wrist")

## Step 5: Basic Motion Analysis

In [ ]:
# Calculate motion magnitude for a specific keypoint
def calculate_motion_magnitude(data, keypoint_name):
    data[f"{keypoint_name}_magnitude"] = data[keypoint_name].diff().abs()
    return data

# Example: Analyse motion magnitude of the Right Hand Wrist
keypoint_data = calculate_motion_magnitude(keypoint_data, "Right Hand Wrist")
print("Motion Magnitude Analysis Sample:")
print(keypoint_data.head())


In [ ]:
# Additional Steps (to be extended as needed):
# - Compare motion data across different files
# - Identify patterns corresponding to backhand/forehand movements
# - Extend analysis for videos once linked to the motion data